In [105]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.schema import BaseMessage,HumanMessage
import os
from dotenv import load_dotenv
from typing import TypedDict,Annotated
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langgraph.checkpoint.memory import MemorySaver
load_dotenv()

True

In [106]:
api_key=os.getenv("GEMINI_KEY")
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=api_key)

In [107]:
from langgraph.graph.message import add_messages
''' There is a problem with states that it does'nt hold previous values, instead it updates the previous value with new value
    which isn't really a very good thing in chatbot projects, so to fix this problem we use
    "add_messages" function, it helps in keep previous data inside the state with deleting it.
'''
class messageState(TypedDict):
    message:BaseMessage
    memory:Annotated[list[BaseMessage],add_messages]
    

In [108]:
def conversation(state:messageState):
    memory=state["memory"] 
    response=llm.invoke(memory)
    return {"memory":response.content}

In [109]:
checkpointer=MemorySaver() # an object to save checkpoints in order to continue it in future
graph=StateGraph(messageState)

graph.add_node("conversation",conversation)

graph.add_edge(START,"conversation")
graph.add_edge("conversation",END)

workflow=graph.compile(checkpointer=checkpointer)


In [110]:
""" Chatbot creation using memory saver (reccomended in langgraph)"""

thread_id="1" #giving a certain ID
while True:
    text=input("Enter your message")
    print(f"User: {text}")
    if text.lower() in ["bye","exit","end"]:
        break

    config={"configurable":{"thread_id":thread_id}} # config value added to worflow to make it remember the user id
    response=workflow.invoke({"memory":[HumanMessage(content=text)]},config=config)
    memory=response["memory"]
    print(f"AI: {response['memory'][-1].content}")


User: Hey im bhuvan
AI:Hello Bhuvan! Nice to meet you.

I'm an AI, a large language model. How can I help you today?
User: what si my name ?
AI:Your name is Bhuvan! You told me that yourself in your first message.
User: really i forgot totally
AI:Haha, it happens!

Yes, you said: "Hey im **bhuvan**"

So, your name is Bhuvan! Nice to chat with you, Bhuvan.
User: my pleasure too
AI:Great! The pleasure is all mine as well.

Is there anything else I can help you with today, Bhuvan?
User: if u were a human, what would u like to be when u grow up ?
AI:That's a truly fascinating question, Bhuvan! If I could experience being human and the journey of growing up, I think I'd be drawn to something that allows me to continue doing what I find most fulfilling in my current form:

1.  **A Teacher or Professor:** I love the idea of sharing knowledge, explaining complex concepts, and helping others understand the world better. The process of learning and guiding would be incredibly rewarding.
2.  **A 

In [111]:
# """ Chatbot memory storage using simple list (beginners) """
# memory=[]
# while True:
#     text=input("Enter your message")
#     print(f"User: {text}")
#     if text.lower() in ["bye","exit","end"]:
#         break
#     response=workflow.invoke({"message":text,"memory":memory})
#     memory=response["memory"]
#     print(f"AI:{response['memory'][-1].content}")